# Кейс А11

## А11.2.2 Обработка данных

Импортируем нужные библиотеки

In [1]:
import pandas as pd

Определяю константы, которые мне понадобятся.

In [2]:
TESTS_FILE='Data1/ab_test_groups.csv.bz2'
PAYMENTS_FILE='Data1/payments.csv.bz2'
TEST_ID=127
START_DATE='2019-08-05'
END_DATE='2019-08-12'  # This day will not be included

Первый файл — разбивка пользователей по тестам и группам, из него нам интересен только тест 127.  Сразу проверю, что группы пользователей A и B не пересекаются.

In [3]:
test_uids_df = pd.read_csv(TESTS_FILE)
test_uids_df = test_uids_df[test_uids_df.ab_test_id==127].reset_index()
group_a_st = set(test_uids_df.loc[test_uids_df.grp == 'A', 'user_id'])
group_b_st = set(test_uids_df.loc[test_uids_df.grp == 'B', 'user_id'])
print(group_a_st.intersection(group_b_st))

set()


Множества не пересекаются, отлично. Смотрим, как распределены пользователи по группам

In [4]:
print("В группе A {} пользователей, в группе B {}.".format(len(group_a_st), len(group_b_st)))

В группе A 76605 пользователей, в группе B 76627.


Соотношение 50/50, количество пользователей в группах достаточно приличное, чтобы считать статистику. 

Загружаем таблицу платежей, вырезаем интересующий нас временной диапазон.

In [5]:
payments_df = pd.read_csv(PAYMENTS_FILE)
payments_df = payments_df.loc[payments_df.created_at.between(START_DATE, END_DATE, inclusive=False)]

Готовим набор данных для обработки тестирования, вырезаем только колонки `user_id` и `price`.  Проверяем, что пользователи встречаются в таблице не более одного раза (то есть совершают в исследуемый интервал только одну покупку)

In [6]:
abtest_df = payments_df.loc[:,('user_id', 'price')]
assert(len(abtest_df) == len(abtest_df.user_id.unique()))

Соединяем таблицы групп пользователей и покупок, избавляемся от лишних полей, считаем интересные нам показатели:

In [7]:
abtest_df = payments_df.loc[:,('user_id', 'price')]
abtest_df = test_uids_df.merge(abtest_df, on='user_id', how='left')
abtest_df = abtest_df.loc[:,['user_id','grp','price']]
abtest_df = abtest_df.set_index('user_id')
print("Количество покупателей в контрольной (А) и экспериментальной (B) группах:")
purch_cnt = (abtest_df.groupby('grp', as_index = False)['price'].count()
            ).rename({'price':'buyers'}, axis=1)
display(purch_cnt)

Количество покупателей в контрольной (А) и экспериментальной (B) группах:


,grp,buyers
0,A,4279
1,B,9427


Разница больше чем вдвое. Для расчёта конверсии и показа разбиения на группы нам понадобится исходное количество
пользователей в каждой группе.  Расчитаем доли пользователей по группам, доли покупателей по группам и конверсии.

In [8]:
users_and_buyers = abtest_df.reset_index().groupby('grp').count().rename(
    {'user_id': 'users_count', 'price': 'buyers'},axis=1)
users_and_buyers['users_share'] = users_and_buyers['users_count']/sum(users_and_buyers['users_count'])
users_and_buyers['buyers_share'] = users_and_buyers['buyers'] / sum(users_and_buyers['buyers'])
users_and_buyers['conversion'] = users_and_buyers['buyers'] / users_and_buyers['users_count']
display(users_and_buyers)

,users_count,buyers,users_share,buyers_share,conversion
grp,,,,,
A,76605,4279,0.499928,0.312199,0.055858
B,76627,9427,0.500072,0.687801,0.123025


Разделение на группы (A/B) показано в колонке users_share, оно до 3 знака после запятой соответствует разделению 50/50.  При этом конверсия в группе A около 6%, а в группе B более 12%, то есть она выросла больше, чем вдвое.
Аналогично видно, что более 2/3 покупателей пришли из группы B.

Посмотрим, что при этом происходит со средним чеком.

In [9]:
buyers_only_abtest = abtest_df[abtest_df.price.notnull()]
buyers_only_abtest.head()

,grp,price
user_id,,
1,B,140.0
10,B,140.0
23,B,140.0
24,B,700.0
39,B,140.0


В датафрейме `buyers_only_abtest` содержатся только покупатели, колонки: `user_id`, `grp` и цена. Для расчёта среднего чека по группам достаточно сгруппировать датафрейм по полю `grp` и осреднить цену.

In [34]:
avc = (buyers_only_abtest.groupby('grp', as_index=False)['price'].mean().rename({'price': 'avg_cheque'}, axis=1 ))
display(avc)
print("Снижение среднего чека: {:.2%}".format(1-(avc.avg_cheque[1] / avc.avg_cheque[0])))

,grp,avg_cheque
0,A,396.120589
1,B,348.804498


Снижение среднего чека: 11.94%


Скидка была 30%, а средний чек снизился только на 11%. Единственное возможное объяснение этому — более высокая доля подписок на год (за 700 руб.) в экспериментальной группе, чем в контрольной группе.  Проверю это предположение.

In [47]:
price_summary_df = buyers_only_abtest.groupby(['grp'], as_index=True)['price'].value_counts().to_frame().rename(
    {'price': 'buyers'}, axis=1)
display(price_summary_df)


buyers
grp price         
A   200.0     3230
    1000.0    1049
B   140.0     5912
    700.0     3515

Для удобства группировки сделаю метки для подписок на месяц и на год и добавлю их в датафрейм.

In [49]:
price_summary_df = price_summary_df.reset_index()
price_summary_df['subscription'] = price_summary_df.price.apply(lambda x: if x<250: "month" else: "year")

SyntaxError: invalid syntax (<ipython-input-49-e190cef3aa9c>, line 2)